# Multi-arm Bandits

假设有 10 个摇臂，在任意时刻 t 只能摇动其中的 1 个，每次摇动之后会获得一个反馈 r。那么问题就是，假如需要摇动 1000 次摇臂，如何使得最后累积的 r 的值最大呢？这就是 Multi-arm Bandits 问题。

In [87]:
# 先创造 10 个随机 [1, 10] 之间的随机整数, 每个整数代表每个摇臂的期望收益
import random

average_rewards = [random.randint(1, 9) for _ in range(0, 10)]
print(average_rewards)


# 同时设定，每个摇臂收益满足正态分布，且方差为 1
from scipy.stats import norm

[1, 2, 9, 1, 9, 3, 4, 4, 3, 4]


## Action-value Methods

这个方法每一个摇臂创建一个 Q(a) 函数，来衡量每一个摇臂摇动之后能获得的价值 r，a 代表具体是哪个摇臂，计算的方法是：

$$
    Q(a) = \frac{\sum_{i=1}^{t-1} r_i I(Action=a)}{\sum_{i=1}^{t-1}I(Action=a)}
$$

应用这个方法，计算以下，上面的 10 个摇臂摇 1000 次以后的 Q 值。

In [11]:
Q = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for n in range(1, 1001):
    for i in range(0, 10):
        Q[i] = norm.rvs(average_rewards[i], 1) / n + (n - 1) * Q[i] / n 

print(Q)

[3.022129494927139, 8.943006690133885, 2.9888146316010675, 1.9989780855226307, 6.002526873274321, 7.992837041004136, 4.9649240459189485, 4.038011320874071, 7.992750798949209, 1.0128159550409008]


上面的计算结果可以看出，最后得到的 Q 里面的值，很接近之前设定的每个摇臂的期望值。但是，如果需要在 1000 次行动之内把收益最大化，上面这种需要 $1000 \times 10 + 1000$ 次的方法就不能使用了。

如果我们用 50 次去探索每个摇臂的 Q 值，然后总是选择 Q 值最大的摇臂摇动，那么可以获得的收益是：

In [15]:
Q = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
total_reward = 0
for n in range(1, 5):
    for i in range(0, 10):
        r = norm.rvs(average_rewards[i], 1)
        Q[i] = r / n + (n - 1) * Q[i] / n
        total_reward += r

max_q = max(Q)
total_reward += 950 * max_q
print(total_reward)

8390.495376772951


通过最初的设定，我们知道，理想中最大的回报是 9000。

### 贪婪方法

在 t 时刻，以 $1 - \epsilon$ 的概率选择摇动在 t 时刻之前 Q(a) 最大的摇臂，同时以 $\epsilon$ 的概率选择所有动作。假设这个 $\epsilon = 0.1$，那么可以得到

In [22]:
Q = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
C = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
total_reward = 0

for i in range(1, 1001):
    epsilon = 0.1
    rand = random.random()
    if rand < epsilon:
        a = random.randint(0, 9)
    else:
        a = Q.index(max(Q))
    
    r = norm.rvs(average_rewards[a], 1)
    total_reward += r
    C[a] += 1
    Q[a] = r / C[a] + (C[a] - 1) / C[a] * Q[a]

print(total_reward)

8351.656668955518


### 乐观的初始值

即设置 Q 的初始值全部为 5，这样更加有利于去探索。

In [25]:
init = 5
Q = [init, init, init, init, init, init, init, init, init, init]
C = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

total_reward = 0
for i in range(1, 1001):
    epsilon = 0.1
    rand = random.random()
    if rand < epsilon:
        a = random.randint(0, 9)
    else:
        a = Q.index(max(Q))
    
    r = norm.rvs(average_rewards[a], 1)
    total_reward += r
    C[a] += 1
    Q[a] = r / C[a] + (C[a] - 1) / C[a] * Q[a]

print(total_reward)

8582.209573379989


### UCB

该方法是另一种鼓励探索的设定。选择摇臂的价值时候，采用如下的公式评估摇臂的价值：

$$
A_t = argmax_{a} \ [Q(a) + c \cdot \sqrt{\frac{ln\ t}{N_t(a)}}]
$$

In [40]:
import math
c = 2
Q = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
C = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
total_reward = 0

for i in range(1, 1001):
    A = [Q[a] + c * math.sqrt(math.log(i) / (C[a] + 1)) for a in range(0, 10)]
    a = A.index(max(A))
    r = norm.rvs(average_rewards[a], 1)
    total_reward += r
    C[a] += 1
    Q[a] = r / C[a] + (C[a] - 1) / C[a] * Q[a]

print(total_reward)

8771.601918799377


### Gradient Bandit

也可以使用梯度来计算收益，计算公式：

$\begin{align*}
H_{t+1}(A_t) & = H_t(A_t) + \alpha(R_t - \overline{R_t})(1 - \pi_t(A_t)) \\
H_{t+1}(a) & = H_t(a) + \alpha(R_t - \overline{R_t})\pi_t(a)
\end{align*}$

上面的式子中 $H_{t}$ 初始化都是 0， $\pi_t$ 的计算公式：

$\pi_t(a)=\frac{e^{H_t{a}}}{\sum_{b=1}^ke^{H_tb}}$

In [142]:
import math

Q = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
C = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
H = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
total_reward = 0

for i in range(1, 1001):
    h_sum = sum([math.exp(h) for h in H])
    p = [math.exp(H[u]) / h_sum for u in range(0, 10)]
    a = p.index(max(p))
    r = norm.rvs(average_rewards[a], 1)
    total_reward += r
    r_t = total_reward / i
    C[a] += 1
    Q[a] = Q[a] + 1 / C[a] * (r - Q[a])
    H[a] = H[a] + 0.1 * (r - r_t) * (1 - p[a])
    TH = [H[q] - 0.1 * (r - r_t) * p[q] for q in range(0, 10)]
    TH[a] = H[a]
    H = TH

print(total_reward)

8968.005769935862
